In [156]:
import pandas as pd
import sklearn

In [157]:
train = pd.read_csv(open('../Data/Kaggle/train.csv'))

In [158]:
test = pd.read_csv(open('../Data/Kaggle/test.csv'))

In [159]:
test['Test'] = 1

In [160]:
combine = pd.merge(train, test, how='outer')

In [161]:
combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 1308
Data columns (total 13 columns):
PassengerId    1309 non-null int64
Survived       891 non-null float64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Test           418 non-null float64
dtypes: float64(4), int64(4), object(5)
memory usage: 117.6+ KB


In [163]:
combine['Is_male']   = combine['Is_female'] = combine['Sex']
combine['Is_male']   = combine['Is_male'].map(lambda x : 1 if x == 'male' else 0)
combine['Is_female'] = combine['Is_female'].map(lambda x : 1 if x == 'female' else 0)

In [164]:
embarked = {'C':0, 'S':1, 'Q':2, '':-1}

In [165]:
combine['Embarked_1'] = combine[combine['Embarked'].notnull()]['Embarked'].map(lambda x : embarked[x])

In [166]:
combine.loc[combine['Embarked'].isnull(), ['Embarked']] = -1

In [167]:
title = ['Dr.', 'Rev.', 'Master.', 'Mrs.', 'Miss.', 'Mr.', 'Mlle.', 'Ms.', 'Capt']
title_dict = {}
for v, k in enumerate(title):
    title_dict[k] = v

In [168]:
combine['Title_Num'] = combine['Name'].map(lambda x : title_dict.get(x.split(' ')[1]), '')
combine['Title']     = combine['Name'].map(lambda x : x.split(' ')[1] if title_dict.get(x.split(' ')[1]) else '')
g_title = combine.groupby(by=['Title', 'Survived'])

In [169]:
def to_int(p_num):
    try:
        int(p_num)
        return False
    except:
        return True

In [171]:
combine[combine['Ticket'].map(lambda x : to_int(x))].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Test,Is_male,Is_female,Embarked_1,Title_Num,Title
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,NaN,1,0,1.0,5.0,Mr.
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,NaN,0,1,0.0,3.0,Mrs.
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,NaN,0,1,1.0,4.0,Miss.
10,11,1.0,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S,NaN,0,1,1.0,4.0,Miss.
12,13,0.0,3,"Saundercock, Mr. William Henry",male,20.0,0,0,A/5. 2151,8.0500,NaN,S,NaN,1,0,1.0,5.0,Mr.


In [172]:
combine.index = combine['PassengerId']

In [173]:
combine[combine['Fare'].isnull()] = combine[(combine['Pclass'] == 3) & (combine['Title'] == 'Mr.') & (combine['Is_male'] == 1)].mean()['Fare']

In [174]:
combine[combine['Embarked_1'].isnull()] = combine[(combine['Pclass'] == 1) & (combine['Title'] == 'Miss.') & (combine['Is_female'] == 1)].median()['Embarked_1']

In [175]:
for i in combine.columns:
    print('{0} = {1}'.format(i, combine[combine[i].isnull()]['PassengerId'].count()))

PassengerId = 0
Survived = 417
Pclass = 0
Name = 0
Sex = 0
Age = 263
SibSp = 0
Parch = 0
Ticket = 0
Fare = 0
Cabin = 1013
Embarked = 0
Test = 889
Is_male = 0
Is_female = 0
Embarked_1 = 0
Title_Num = 47
Title = 0


In [176]:
used_columns = ['Pclass', 'Fare', 'Is_male', 'Is_female', 'Survived', 'Test']

In [177]:
combine = combine[used_columns]

In [178]:
combine.columns

Index(['Pclass', 'Fare', 'Is_male', 'Is_female', 'Survived', 'Test'], dtype='object')

In [180]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [181]:
X_train = combine[combine['Test'].isnull()][used_columns]
X_test  = combine[combine['Test'].notnull()][used_columns]
del X_train['Test']
del X_test['Test']

In [182]:
y_train = np.asarray(X_train['Survived'], dtype="|S6")
y_test = np.asarray(X_test['Survived'], dtype="|S6")

In [183]:
del X_train['Survived']
del X_test['Survived']

In [184]:
rfe = model.fit(X_train, y_train)

c:\users\anikolaev\documents\github\machinelearning\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [185]:
r = rfe.predict(X_test)

In [186]:
r

array([b'1.0', b'1.0', b'0.0', b'1.0', b'0.0', b'0.0', b'1.0', b'0.0',
       b'1.0', b'0.0', b'1.0', b'0.0', b'0.0', b'0.0', b'1.0', b'0.0',
       b'1.0', b'1.0', b'0.0', b'0.0', b'1.0', b'1.0', b'0.0', b'0.0',
       b'1.0', b'0.0', b'1.0', b'0.0', b'1.0', b'0.0', b'0.0', b'0.0',
       b'0.0', b'0.0', b'1.0', b'1.0', b'0.0', b'0.0', b'1.0', b'1.0',
       b'0.0', b'0.0', b'0.0', b'0.0', b'0.0', b'1.0', b'1.0', b'0.0',
       b'0.0', b'0.0', b'1.0', b'1.0', b'0.0', b'0.0', b'1.0', b'1.0',
       b'0.0', b'0.0', b'0.0', b'0.0', b'0.0', b'1.0', b'0.0', b'0.0',
       b'0.0', b'1.0', b'1.0', b'1.0', b'1.0', b'0.0', b'0.0', b'1.0',
       b'1.0', b'0.0', b'1.0', b'0.0', b'1.0', b'0.0', b'0.0', b'1.0',
       b'0.0', b'1.0', b'0.0', b'0.0', b'0.0', b'0.0', b'0.0', b'0.0',
       b'1.0', b'1.0', b'1.0', b'0.0', b'1.0', b'0.0', b'1.0', b'0.0',
       b'0.0', b'0.0', b'1.0', b'0.0', b'1.0', b'0.0', b'1.0', b'0.0',
       b'0.0', b'0.0', b'1.0', b'0.0', b'0.0', b'0.0', b'0.0', b'0.0',
      